# 06 - Experiment NotebookUse this notebook to quickly test different approaches.

## Quick Setup

In [ ]:
import pandas as pdimport numpy as npimport xgboost as xgbfrom sklearn.model_selection import cross_val_score, TimeSeriesSplitfrom pathlib import Pathimport warningswarnings.filterwarnings('ignore')BASE_PATH = Path('..')FEATURES_PATH = BASE_PATH / 'features_v2'# Load datadf = pd.read_csv(FEATURES_PATH / 'forecast_sku_weekly_H1.csv')print(f"Loaded: {len(df):,} rows")

## Experiment: Different Hyperparameters

In [ ]:
# Quick feature prepdf['sku_encoded'] = pd.factorize(df['sku'])[0]feature_cols = ['sku_encoded', 'week_num']if 'lag1_quantity' in df.columns:    feature_cols.extend(['lag1_quantity', 'lag2_quantity'])df_clean = df.dropna(subset=feature_cols)X = df_clean[feature_cols]y = df_clean['weekly_revenue']print(f"Training data: {len(X):,} samples")

In [ ]:
# Test different configurationsconfigs = [    {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1},    {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1},    {'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.05},]results = []for cfg in configs:    model = xgb.XGBRegressor(**cfg, random_state=42, n_jobs=-1)        # Time series cross-validation    tscv = TimeSeriesSplit(n_splits=3)    scores = cross_val_score(model, X, y, cv=tscv, scoring='neg_mean_absolute_error')        results.append({        **cfg,        'cv_mae': -scores.mean(),        'cv_std': scores.std()    })    print(f"{cfg} -> MAE: {-scores.mean():,.0f} (+/- {scores.std():,.0f})")results_df = pd.DataFrame(results)print("\nBest config:")print(results_df.loc[results_df['cv_mae'].idxmin()])

## NotesAdd your experiment observations here:- ...- ...